   # Part10: Run Notebooks


In [1]:
import os
import sys
import time
import datetime
import touch
import inspect
import numpy
import pandas as pd
print(sys.version_info)

sys.version_info(major=3, minor=9, micro=2, releaselevel='final', serial=0)


In [2]:
mlist = list(filter(lambda x: inspect.ismodule(x[1]), locals().items()))
vi = sys.version_info
print("version {0}.{1}.{2} of Python".format(vi.major, vi.minor, vi.micro))
for name, mod in mlist:
    mname = name
    if name.startswith("__"):
        continue
    if hasattr(mod, "__version__"):
        mname = name
        if hasattr(mod, "__path__"):
            mname = os.path.split(mod.__path__[0])[1]
        print("version {1} of {0} as {2} ".format(mname, name, mod.__version__))
    elif hasattr(mod, "__file__") and "site-packages" in mod.__file__:
        print("No __version__ for {0} as {1}".format(mname, name))
del mod
del name

version 3.9.2 of Python
No __version__ for touch as touch
version numpy of numpy as 1.20.1 
version pd of pandas as 1.2.3 


In [3]:
# to get the names of notebook files in a directory
def get_project_notebooks():
    from nbformat import read, NO_CONVERT
    import os
    import datetime
    proj_nbs = []
    nbs = [f for f in os.listdir() if f.endswith(".ipynb")]
    part_nbs = [f for f in nbs if f.startswith("Part")]
    return(part_nbs)


In [4]:
# print the source code for a cell
def print_source(cell):
    print("type cell= {0}".format(type(cell)))
    src = cell["source"]
    lines = src.split("\n")
    print("{0} lines".format(len(lines)))
    for i,line in enumerate(lines):
        print("({0}){1}".format(i, line))

# run notebook code
def run_nb_code(nbfile):
    from nbformat import read, NO_CONVERT
    import os
    import datetime
    res = "Start time: " + str(datetime.datetime.now())
    with open(nbfile) as fp:
        notebook = read(fp, NO_CONVERT)
    cells = notebook['cells']
    code_cells = [c for c in cells if c['cell_type'] == 'code']
    for i, cell in enumerate(code_cells):
        src = cell['source']
        #print("{0}<{1}>".format(li, line))                
        try:
            exec(src, globals(), locals())
        except Exception as e:
            print("Error on cell {0}\n".format(i))
            print(src)
            print_source(cell)
            exc_type, exc_obj, exc_tb = sys.exc_info()
            import traceback
            traceback.print_tb(exc_tb)
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            print(exc_type, fname, exc_tb.tb_lineno)
            return((exc_type, exc_obj, exc_tb, fname))
    res += "end: " + str(datetime.datetime.now())
    return res
# run notebook code
def get_first_cell_line(nbfile):
    from nbformat import read, NO_CONVERT
    import os
    import datetime
    res = "Start time: " + str(datetime.datetime.now())
    with open(nbfile) as fp:
        notebook = read(fp, NO_CONVERT)
    cells = notebook['cells']
    if not cells:
        return ""
    lines = cells[0]["source"].split("\n")
    if not lines:
        return ""
    return lines[0]


In [5]:
def get_run_order(nb_names, verbosity=0):
  tups = []
  not_run = []
  
  for nb_name in nb_names:
    if verbosity > 0:
      print(nb_name)
    name_parts = nb_name.split("_")
    if len(name_parts) < 2:
      not_run.append(("name doesn't start with 'Part'", nb_name))
      continue
    elif name_parts[1] == "99":
      not_run.append(("runall", nb_name))
      continue
    try:
      id = int(name_parts[1])
      tups.append((id, nb_name))
    except:
      not_run.append("can't convert {0} to int".format(name_parts[1]), nb_name)
  tups.sort()
  to_run = [x[1] for x in tups]
  return to_run, not_run
  
  
nb_names = get_project_notebooks()

to_run, not_run = get_run_order(nb_names, verbosity=0)
print(to_run)

['Part_0_Setup.ipynb', 'Part_1_Intro_Overview.ipynb', 'Part_2_DataSources.ipynb', 'Part_3_Fred.ipynb', 'Part_4_Weekly_Claims.ipynb', 'Part_5_UMich_Sentiment.ipynb', 'Part_6_GoogleTrends.ipynb', 'Part_7_Plot_Claims.ipynb', 'Part_8_Combine_Datasets.ipynb']


In [9]:
%%capture --no-stderr nb_out
def extract_part(name):
  parts = name.split("_")
  try:
    part = int(parts[1])
    return part
  except:
    exc_type, exc_obj, exc_tb = sys.exc_info()
    fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
    print(exc_type, fname, exc_tb.tb_lineno)     
    
for nb_name in to_run:
    part = extract_part(nb_name)
    if part < 0:
      continue
    sys.stderr.write("{0}  <{1}>\n".format(nb_name, datetime.datetime.now()))
    %run $nb_name
    sys.stderr.write("  done  <{0}>\n".format(nb_name, datetime.datetime.now()))    

Part_0_Setup.ipynb  <2021-11-02 11:25:59.690503>
  done  <Part_0_Setup.ipynb>
Part_1_Intro_Overview.ipynb  <2021-11-02 11:25:59.693495>
  done  <Part_1_Intro_Overview.ipynb>
Part_2_DataSources.ipynb  <2021-11-02 11:25:59.696032>
  done  <Part_2_DataSources.ipynb>
Part_3_Fred.ipynb  <2021-11-02 11:25:59.697990>
  done  <Part_3_Fred.ipynb>
Part_4_Weekly_Claims.ipynb  <2021-11-02 11:26:18.799586>
  done  <Part_4_Weekly_Claims.ipynb>
Part_5_UMich_Sentiment.ipynb  <2021-11-02 11:26:26.377131>
  done  <Part_5_UMich_Sentiment.ipynb>
Part_6_GoogleTrends.ipynb  <2021-11-02 11:26:31.142072>
  done  <Part_6_GoogleTrends.ipynb>
Part_7_Plot_Claims.ipynb  <2021-11-02 11:28:09.827292>
  done  <Part_7_Plot_Claims.ipynb>
Part_8_Combine_Datasets.ipynb  <2021-11-02 11:28:10.359765>


NameError: name 'fulldf' is not defined

NameError: name 'fulldf' is not defined

In [7]:
print(nb_out.stdout)

version 3.9.2 of Python
version 1.20.1 of numpy
version 1.2.3 of pd
version 1.20.1 of np
version 3.3.4 of mpl
series: PAYEMS, obs_start: 2010-01-01, obs_end: 2021-11-02
get_series_all_releases <2021-11-02 11:05:34.886080
(1107, 3)
rows= 141
series: USPRIV, obs_start: 2010-01-01, obs_end: 2021-11-02
get_series_all_releases <2021-11-02 11:05:35.468883
(1107, 3)
rows= 141
series: NPPTTL, obs_start: 2010-01-01, obs_end: 2021-11-02
get_series_all_releases <2021-11-02 11:05:36.028408
(1521, 3)
rows= 141
series: ICSA, obs_start: 2010-01-01, obs_end: 2021-11-02
get_series_all_releases <2021-11-02 11:05:36.646222
(3444, 3)
rows= 617
series: CCSA, obs_start: 2010-01-01, obs_end: 2021-11-02
get_series_all_releases <2021-11-02 11:05:37.668904
(3817, 3)
rows= 616
series: JTS1000JOL, obs_start: 2010-01-01, obs_end: 2021-11-02
get_series_all_releases <2021-11-02 11:05:38.765210
(930, 3)
rows= 140
series: JTS1000HIL, obs_start: 2010-01-01, obs_end: 2021-11-02
get_series_all_releases <2021-11-02 11:05:

In [8]:
import datetime
print(datetime.datetime.now())

2021-11-02 11:07:10.488786
